# Análisis de datos en Cancelaciones

## Consigna

El dataset "cancelaciones" preparado para predecir cancelaciones en reservas de cuartos de hotel. La variable target es "booking_status", donde el valor 1 indica que la reserva fue cancelada. Si elegís este dataset, se solicita lo siguiente:

<ol>
  <li>Analizar el dataset para detectar posibles correlaciones, outliers, datos inválidos, columnas no necesarias, etc. Se pueden usar funciones descriptivas, gráficos, etc.</li>
  <li>Preprocesar el dataset y generar conjuntos de entrenamiento y testeo.</li>
  <li>Ajustar tres modelos (regresión, árbol y KNN) para predecir la variable target. Evaluar cada modelo e indicar cual seleccionarían y por qué.</li>
  <li>Determinar cuales son los predictores mas importantes de cara a estimar el valor de la variable target.</li>
  <li>Seleccionar un predictor y generar tantos modelos como valores admite dicho predictor (el tipo de predictor, regresión, árbol o KNN, lo deben seleccionar ustedes, puede ser cualquiera). Guardar cada modelo en una lista o diccionario, de manera tal de poder acceder a cada modelo por el valor asociado del predictor seleccionado. </li>
  <li>Crear una función que tome como parámetros la lista o diccionario anterior y un dataframe predictores. La función debe devolver un vector o dataframe con los resultados de las predicciones. Para ello, internamente, la función debe seleccionar el modelo correcto de la lista o diccionario para realizar la predicción.</li>
</ol>

## Librerias

In [ ]:
library(dplyr)
library(tidyverse)
library(ggplot2)
library(caret)
library(glue)


## Funciones

In [ ]:
library(dplyr)

# Detecta si el tipo de variable es continuo
is.continuous <- function(x) {
    return(is.double(x))
}

# Detecta si el tipo de variable es discreto
is.discrete <- function(x) {
    return(is.factor(x) || is.integer(x))
}

# Detecta si el tipo de variable admite una comparación de correlación
is.correlation.applicable <- function(x) {
    return(is.numeric(x) || is.logical(x))
}

# Detecta si el tipo de variable admite normalización
is.normalization.applicable <- function(x) {
    return(is.numeric(x))
}

# Devuelve todos los outliers apartados a mas de 3 desvíos del promedio
find_outliers <- function(datos_columna) {
    promedio <- mean(datos_columna)
    desvio <- sd(datos_columna)
    limite_bajo <- promedio - 3 * desvio
    limite_alto <- promedio + 3 * desvio
    return(which(!dplyr::between(datos_columna, limite_bajo, limite_alto)))
}

# Normaliza la columna y la devuelve como parámetro
normalize_column <- function(datos_columna) {
    promedio <- mean(datos_columna)
    desvio <- sd(datos_columna)
    return(as.double(lapply(
        datos_columna,
        function(x) {
            return((x - promedio) / desvio)
        }
    )))
}

# Realiza una predicción a partir de una modelo entrenado y evalúa partir de datos de prueba
# Imprime la predicción y los parametros de salida
evaluate_model <- function(nombre_modelo, modelo_entrenado, datos_test) {
    prediccion <- predict(modelo_entrenado, newdata = datos_test)
    precision <- caret::posPredValue(prediccion, datos_test$booking_status, positive = 1)
    recall <- caret::sensitivity(prediccion, datos_test$booking_status, positive = 1)
    F1 <- (2 * precision * recall) / (precision + recall)

    resultado <- list(
        "prediccion" = prediccion,
        "result" = list(
            "precision" = precision,
            "recall" = recall,
            "F1" = F1
        )
    )

    print(nombre_modelo)
    print(caret::confusionMatrix(prediccion, datos_test$booking_status))
    print(resultado$result)
}

# Realiza una predicción a partir de un modelo entrenado
# y agrega la predicción como columna al dataframe enviado por parámetro
predict_dataframe <- function(trained_models, datos_test, min_room_type_reserved, max_room_type_reserved) {
    datos_particiones_test <- datos_test[FALSE, ]
    for (room_type in min_room_type_reserved:max_room_type_reserved) {
        index <- room_type + 1
        datos_particion_test <- datos_test %>% filter(datos_test$room_type_reserved == room_type)
        prediccion <- predict(trained_models[index], newdata = datos_particion_test)
        datos_particion_test$prediccion <- prediccion[[1]]
        datos_particiones_test <- datos_particiones_test %>% rbind(datos_particion_test)
    }

    return(
        datos_particiones_test[order(as.numeric(row.names(datos_particiones_test))), ]
    )
}


## Lectura de datos

Se leen los datos desde un csv y ser convierten las columnas para mejorar su tipificación y manejo.

In [ ]:
datos <- read.csv("../sources/cancelaciones.csv")

# Se leen los limites del campo "room_type_reserved" que en pasos
# posteriores serán usados para generar los modelos individuales
max_room_type_reserved <- max(datos$room_type_reserved)
min_room_type_reserved <- min(datos$room_type_reserved)

datos <- datos %>% mutate(
    booking_status = as.factor(booking_status),
    type_of_meal_plan = as.factor(type_of_meal_plan),
    required_car_parking_space = as.logical(required_car_parking_space),
    room_type_reserved = as.factor(room_type_reserved),
    market_segment_type = as.factor(market_segment_type),
    repeated_guest = as.logical(repeated_guest),
    arrival_full_date = as.Date(str_glue("{arrival_year}-{arrival_month}-{arrival_date}")),
    arrival_year = NULL,
    arrival_month = NULL,
    arrival_date = NULL
)


## Outliers

Detección y filtrado de outliers. Adicionalmente, se chequea el formato de la fecha que en algunas filas esta incompleta.

El criterio de detección de outliers es de 3 desvíos de distancia como máximo. 

Teniendo en cuenta la cantidad de datos recolectados, todas las filas que estén fuera de este criterio en alguna de las columnas, son desestimadas del análisis.

In [ ]:
datos <- datos %>% filter(!row_number() %in% find_outliers(datos$lead_time))
datos <- datos %>% filter(!row_number() %in% find_outliers(datos$avg_price_per_room))
datos <- datos %>% filter(!row_number() %in% find_outliers(datos$no_of_previous_bookings_not_canceled))
datos <- datos %>% filter(!row_number() %in% find_outliers(datos$no_of_previous_cancellations))
datos <- datos %>% filter(!row_number() %in% find_outliers(datos$no_of_adults))
datos <- datos %>% filter(!row_number() %in% find_outliers(datos$no_of_children))
datos <- datos %>% filter(!row_number() %in% find_outliers(datos$no_of_weekend_nights))
datos <- datos %>% filter(!row_number() %in% find_outliers(datos$no_of_week_nights))
datos <- datos %>% filter(!row_number() %in% find_outliers(datos$no_of_special_requests))
datos <- datos %>% filter(!is.na(arrival_full_date))


## Gráficos

Plotea gráficos para las variables discretas y continuas por separado. Para las primeras hace gráficos de barra y para las segundas, de puntos.

Algunos gráficos no son relevantes, pero son generados como parte del conjunto.

In [ ]:
variables_discretas <- datos %>% dplyr::select(where(is.discrete))
variables_discretas_colnames <- colnames(variables_discretas)
variables_continuas <- datos %>% dplyr::select(where(is.continuous))
variables_continuas_colnames <- colnames(variables_continuas)

for (index in seq_along(variables_discretas_colnames)) {
    print(ggplot(datos, aes(variables_discretas[, index])) +
        geom_bar() +
        xlab(variables_discretas_colnames[index])) 
    print(ggplot(datos, aes(variables_discretas[, index])) +
        geom_boxplot(
            outlier.colour = "black", outlier.shape = 16,
            outlier.size = 2, notch = FALSE
        ) +
        xlab(variables_discretas_colnames[index]))
}

for (index in seq_along(variables_continuas_colnames)) {
    print(ggplot(datos, aes(x = variables_continuas[, index], y = booking_status)) +
        geom_point() +
        xlab(variables_continuas_colnames[index])) 
    print(ggplot(datos, aes(variables_continuas[, index])) +
        geom_boxplot(
            outlier.colour = "black", outlier.shape = 16,
            outlier.size = 2, notch = FALSE
        ) +
        xlab(variables_continuas_colnames[index]))
}


## Análisis de Correlación

El análisis de correlación indica que no hay variables notoriamente dependientes.

Como parte del proceso, se habían agregado variables como suma de otras variables (por ejemplo: n° personas = n° niños + n° adultos), se retiraron porque generaban una gran correlación entre ellas por poder derivarse linealmente de otras columnas. 

In [ ]:
analisis_correlacion <- cor(datos %>% dplyr::select(where(is.correlation.applicable)))
print(analisis_correlacion)


## Descomposición de columnas

Se descomponen las columnas de tipo factor de cardinalidad "n" en "n" columnas de tipo lógico para poder ser utilizadas en los modelos de predicción posteriores.

Una vez descompuestas las columnas, son eliminadas del dataset.

Particularmente type_of_meal_plan y market_segment_type no son descompuestos porque se encontró en etapas posteriores que producen un modelo errático, por lo que directamente son ignoradas.

In [ ]:
datos <- datos %>% mutate(
    arrival_day_of_the_week = as.factor(wday(arrival_full_date)),
    arrival_day_of_the_week_1 = as.logical(arrival_day_of_the_week == 1),
    arrival_day_of_the_week_2 = as.logical(arrival_day_of_the_week == 2),
    arrival_day_of_the_week_3 = as.logical(arrival_day_of_the_week == 3),
    arrival_day_of_the_week_4 = as.logical(arrival_day_of_the_week == 4),
    arrival_day_of_the_week_5 = as.logical(arrival_day_of_the_week == 5),
    arrival_day_of_the_week_6 = as.logical(arrival_day_of_the_week == 6),
    arrival_day_of_the_week_7 = as.logical(arrival_day_of_the_week == 7),
    room_type_reserved_0 = as.logical(room_type_reserved == 0),
    room_type_reserved_1 = as.logical(room_type_reserved == 1),
    room_type_reserved_2 = as.logical(room_type_reserved == 2),
    room_type_reserved_3 = as.logical(room_type_reserved == 3),
    room_type_reserved_4 = as.logical(room_type_reserved == 4),
    room_type_reserved_5 = as.logical(room_type_reserved == 5),
    room_type_reserved_6 = as.logical(room_type_reserved == 6),
    lead_time = normalize_column(lead_time),
    avg_price_per_room = normalize_column(avg_price_per_room),
    no_of_special_requests = normalize_column(no_of_special_requests),
    no_of_adults = normalize_column(no_of_adults),
    no_of_children = normalize_column(no_of_children),
    no_of_weekend_nights = normalize_column(no_of_weekend_nights),
    no_of_week_nights = normalize_column(no_of_week_nights),
    no_of_previous_cancellations = NULL,
    no_of_previous_bookings_not_canceled = NULL,
    arrival_day_of_the_week = NULL,
    type_of_meal_plan = NULL,
    market_segment_type = NULL,
    arrival_full_date = NULL
)

datos_todos <- datos %>% mutate(
    room_type_reserved = NULL
)


## Evaluación de modelos

Los modelos de regresión lineal, KNN y árbol son entrenados y evaluados.

Los datos de entrenamiento y de prueba se crean aleatorimente sin un criterio específico.

La evaluación del modelo es impresa en la consola.

El criterio de desición es exclusivamente el paramétro F1 da una idea del rendimiento promedio del modelo en casos positivos y negativos acertados.

Se determina que el modelo que mejor se ajusta es KNN, por tener un valor de F1 0.69, el mayor entre los 3 modelos evaluados.

In [ ]:
set.seed(0)
indices_train <- sample(seq_len(nrow(datos_todos)), 0.8 * nrow(datos_todos))
datos_todos_train <- datos_todos[indices_train, ]
datos_todos_test <- datos_todos[-indices_train, ]

train_model_lm <- train(
    booking_status ~ .,
    data = datos_todos_train,
    method = "glm",
    family = "binomial",
    trControl = trainControl(method = "none")
)

evaluate_model("Regresion Lineal", train_model_lm, datos_todos_test)

train_model_knn <- train(
    booking_status ~ .,
    data = datos_todos_train,
    method = "knn",
    trControl = trainControl(method = "none"),
    tuneGrid = expand.grid(k = 5)
)

evaluate_model("KNN", train_model_knn, datos_todos_test)

train_model_tree <- train(
    booking_status ~ .,
    data = datos_todos_train,
    method = "rpart"
)

evaluate_model("Árbol", train_model_tree, datos_todos_test)


## Evaluación de predictores

Reutilizando el modelo lineal creado en el paso anterior, se analiza la influencia de cada uno de los predictores. 

Si bien no ha sido el método seleccionado como predictor, la regresión lineal nos sirve igualmente para medir el grado de influencia de los predictores.

Siendo que los predictores están normalizados, el estimador de cada uno nos da una idea de cuanto influye cada predictor en particular.

Las columnas más significativas son "arrival_day_of_the_week", "no_of_week_nights" y "repeated_guest". 

El resultado de las columnas de "room_type_reserved" no es representativo porque tiene el mismo valor para todas las opciones.

Por otra parte, "arrival_day_of_the_week" muestra que hay días de la semana que son más críticos que otros en cuanto a la influencia positiva o negativa que puede llegar a haber en la cancelación de un cliente.

Puntualmente "arrival_day_of_the_week_7" no se evalúa porque el modelo no puede separarlo del resto de los días debido a que es una dependencia lineal de los otros 6 días.

In [ ]:
summary(train_model_lm)


## Modelos particionados

Genera un modelo distintos para cada valor posible dentro de "room_type_reserved". 

Permite ver el comportamiento de los datos separados en subconjuntos.

Se realiza e imprime una evaluación de cada uno de los subconjuntos de datos.

Los datos de entrenamiento y de prueba se crean aleatorimente sin un criterio específico para cada uno de los subconjuntos.

Los subconjuntos correspondientes a "room_type_reserved" == 5 y "room_type_reserved" == 6 tienen un tamaño insuficiente y no representativo, por lo que los modelos no pueden ser calculados y la salida producida no es representativa en absoluto.

Finalmente, se guardan todas las filas de prueba en un dataframe junto con la salida de la predicción para cada columna. Al algoritmo predice cada fila con el subconjunto de datos coincidente con su "room_type_reserved".

Para el caso puntual, se observa que la división según "room_type_reserved" provoca mejoras en el parámetro F1 para "room_type_reserved" == 0 por un 0.08, que es el caso más significativo por tener la mayor cantidad de reservas. En caso de continuar el análisis, sugeriría seguir esta línea de división del modelo según "room_type_reserved".

In [ ]:
datos$prediccion <- FALSE
datos_particiones_test <- datos[FALSE, ]

modelos_particiones <- c()

for (index in min_room_type_reserved:max_room_type_reserved) {
    datos_particion <- datos %>% filter(datos$room_type_reserved == index)

    indices_particion_train <- sample(seq_len(nrow(datos_particion)), 0.8 * nrow(datos_particion))
    datos_particion_train <- datos_particion[indices_particion_train, ]
    datos_particion_test <- datos_particion[-indices_particion_train, ]

    train_model_particion <- train(
        booking_status ~ .,
        data = datos_todos_train,
        method = "knn",
        trControl = trainControl(method = "none"),
        tuneGrid = expand.grid(k = 5)
    )

    evaluate_model(
        glue("Particion: room_type_reserved == {index}"),
        train_model_particion,
        datos_particion_test
    )

    modelos_particiones <- c(modelos_particiones, list(train_model_particion))
    datos_particiones_test <- datos_particiones_test %>% rbind(datos_particion_test)
}

resultados <- predict_dataframe(
    modelos_particiones,
    datos_particiones_test,
    min_room_type_reserved,
    max_room_type_reserved
)

head(resultados)


## Conclusiones

Se implementaron distintas técnicas de preprocesamiento para manejar outliers, normalizar los datos y optimizar las variables, buscando un modelo más preciso. 
La evaluación comparativa entre los modelos de regresión, árbol de decisión y KNN nos permitió identificar el modelo más efectivo en términos de precisión y F1-score, resultando en un predictor robusto de la cancelación de reservas.

Se determinó que el modelo KNN (K-Nearest Neighbors) fue el mejor en términos de precisión general para predecir cancelaciones. Este algoritmo clasifica cada instancia en función de la clase predominante entre sus "vecinos" más cercanos. En este caso, el modelo KNN encontró patrones relevantes entre las características de los clientes y las reservas.

Durante el análisis, se observó que la división del conjunto de datos según el tipo de habitación reservada mejoró el rendimiento del modelo. Para las reservas más comunes, la métrica F1 se incrementó en 0.08.